In [ ]:
########################### DATASET PROCESSING ####################################

import torch
import numpy as np
import torchvision
from torchvision import transforms
from numpy import genfromtxt
import os
from tqdm import tqdm
from torch.utils.data import DataLoader

REBUILD_DATA = True
maxAX=1.997222249219917
minAX=-0.4888889218044315
maxAY=1.634722245672321
minAY=-1.252777853006882
maxAZ=1.463888933022928
minAZ=-0.7916666716198171
maxGX=5.320330619812012
minGX=-3.276986598968506
maxGY=3.133127927780151
minGY=-4.782769203186035
maxGZ=1.985922932624817
minGZ=-2.441933870315552


class HAR():
    
    labels = []
    j= 0 #to loop over labels file 
    exp_user =[]
    sample = [] # to put all six readings then add in the big list (all_data)
    sample_label=[] # to add the sample array and label in one array
    
    
    text_path = 'labels.txt'
    exp_id = []
    user_id = []
    act_id = []
    beg_id = []
    end_id = []
    
    ######### FOR NORMALIZATION #########
    total = 19082*3
    sum=0
    mean=0
    std=0
   
    

    test_user_ids = [2, 4, 9, 10, 12, 13, 18, 20, 24];   #check for those ids for test data
    # for train data
    acc_x =[]
    acc_y =[]
    acc_z =[]

    gyro_x =[]
    gyro_y =[]
    gyro_z =[]

    exp_user = []  
    # for test data
    t_acc_x =[]
    t_acc_y =[]
    t_acc_z =[]
    t_gyro_x =[]
    t_gyro_y =[]
    t_gyro_z =[]
    # for putting all data after reading from files abl ma a2semha to training and testing
    all_x = []
    all_y = []
    all_z = []
    all_gx = []
    all_gy = []
    all_gz = []
    values = []
   
    labels = []
    test_labels = []

    all_data = []
    test_all_data = []
    segment_size = 128 # how many samples in each segment
    
    ax_data=[]
    ay_data=[]
    az_data=[]
    gx_data=[]
    gy_data=[]
    gz_data=[]
    
    
    ax_test_data=[]
    ay_test_data=[]
    az_test_data=[]
    gx_test_data=[]
    gy_test_data=[]
    gz_test_data=[]
    
    
    
    def data_processing(self):
        with open(self.text_path) as f:
            text_data = f.readlines()
            for line_index in range(len(text_data)):
                line = text_data[line_index].strip('\n')
                words = line.split()
                self.exp_id.append(words[0])
                self.user_id.append(words[1])
                self.act_id.append(words[2])
                self.beg_id.append(words[3])
                self.end_id.append(int(words[4],10))
                self.exp_user.append((words[0],words[1]))  #pairs of exp and user ids
        self.exp_user = list(dict.fromkeys(self.exp_user))
        for i in range(len(self.exp_user)):  # loop over the files (experiments)
            self.all_x = []
            self.all_y = []
            self.all_z = []          #RESET
            self.all_gx = []
            self.all_gy = []
            self.all_gz = []
            if(int(self.exp_user[i][0],10))<10:
                acc_path = 'acc_exp0'+ self.exp_user[i][0]
                gyro_path = 'gyro_exp0'+ self.exp_user[i][0]
            else:
                acc_path = 'acc_exp'+ self.exp_user[i][0]
                gyro_path = 'gyro_exp'+ self.exp_user[i][0]
                
            if(int(self.exp_user[i][1],10))<10:
                acc_path = acc_path+'_user0'+ self.exp_user[i][1]+'.txt'
                gyro_path = gyro_path+'_user0'+ self.exp_user[i][1]+'.txt'
                
            else:
                acc_path = acc_path+'_user'+ self.exp_user[i][1]+'.txt'
                gyro_path = gyro_path+'_user'+ self.exp_user[i][1]+'.txt'
                
            with open(acc_path) as f: #open acc file and put all the content in all_x
                text_data = f.readlines()
                for line_index in range(len(text_data)):
                    line = text_data[line_index].strip('\n')
                    words = line.split()
                    ########## NORMALIZE DATA ########
                    
                    accX = (float(words[0]) - minAX) / (maxAX - minAX)
                    accY = (float(words[1]) - minAY) / (maxAY - minAY)
                    accZ = (float(words[2]) - minAZ) / (maxAZ - minAZ)
                    self.all_x.append(accX)
                    self.all_y.append(accY)
                    self.all_z.append(accZ)
                    
                    
            with open(gyro_path) as f: #open gyro file and put all the content in all_gx
                text_data = f.readlines() 
                for line_index in range(len(text_data)): 
                    line = text_data[line_index].strip('\n')
                    words = line.split()
                    ########## NORMALIZE DATA ########
                    
                    gyroX = (float(words[0]) - minGX) / (maxGX - minGX)
                    gyroY = (float(words[1]) - minGY) / (maxGY - minGY)
                    gyroZ = (float(words[2]) - minGZ) / (maxGZ - minGZ)
                    self.all_gx.append(gyroX)
                    self.all_gy.append(gyroY)
                    self.all_gz.append(gyroZ)
                   
        
            while (self.j<1214) and (self.exp_user[i][0]== self.exp_id[self.j]) and (self.exp_user[i][1]== self.user_id[self.j]):
                k = int(self.beg_id[self.j],10)-1
                sampleAX=[]
                sampleAY=[]
                sampleAZ=[]
                sampleGX=[]
                sampleGY=[]
                sampleGZ=[]
               
                if int(self.act_id[self.j],10)<7:# only take first 6 labels
                    while k+self.segment_size<=self.end_id[self.j]:# start the new segment
                        b = k
                        while b <k+self.segment_size: # fill up the segment
                            sampleAX.append(float(self.all_x[b]))
                            sampleAY.append(float(self.all_y[b]))
                            sampleAZ.append(float(self.all_z[b]))
                            sampleGX.append(float(self.all_gx[b]))
                            sampleGY.append(float(self.all_gy[b]))
                            sampleGZ.append(float(self.all_gz[b]))
                            b+=1
                            
                        if int(self.user_id[self.j],10) in self.test_user_ids: #checking for train or test
                           
                            #self.sample_label.append(self.values)
                            self.test_labels.append(np.eye(6)[int(self.act_id[self.j],10)-1]) #adding label with segment
                            
                            self.ax_test_data.append(sampleAX)
                            self.ay_test_data.append(sampleAY)
                            self.az_test_data.append(sampleAZ)
                            self.gx_test_data.append(sampleGX)
                            self.gy_test_data.append(sampleGY)
                            self.gz_test_data.append(sampleGZ)
                            
                            #print(self.sample_label)
                            #print(self.sample_label[0])
                            #print(self.sample_label[1])
                            #print(len(self.values))
                            
                        else:
                            self.labels.append(np.eye(6)[int(self.act_id[self.j],10)-1]) #adding label with segment
                            
                            self.ax_data.append(sampleAX)
                            self.ay_data.append(sampleAY)
                            self.az_data.append(sampleAZ)
                            self.gx_data.append(sampleGX)
                            self.gy_data.append(sampleGY)
                            self.gz_data.append(sampleGZ)
                            
                            
                        sampleAX = []
                        sampleAY = []
                        sampleAZ = []
                        sampleGX = []
                        sampleGY = []
                        sampleGZ = []    
                        
                        k = k+self.segment_size # to start where the last segment ended
                        self.sample = []
                        self.sample_label = []
                        self.values = []
                        
                self.j+=1
        
        
        #print(len(self.test_labels))
        # SHUFFLING SEGMENTS
        #np.random.shuffle(self.all_data)
        #np.random.shuffle(self.test_all_data)
        np.save("training_data_ax.npy",self.ax_data)
        np.save("training_data_ay.npy",self.ay_data)
        np.save("training_data_az.npy",self.az_data)
        
        np.save("training_data_gx.npy",self.gx_data)
        np.save("training_data_gy.npy",self.gy_data)
        np.save("training_data_gz.npy",self.gz_data)
        
        np.save("training_labels.npy",self.labels)
        
        
        
        np.save("testing_data_ax.npy",self.ax_test_data)
        np.save("testing_data_ay.npy",self.ay_test_data)
        np.save("testing_data_az.npy",self.az_test_data)
        
        np.save("testing_data_gx.npy",self.gx_test_data)
        np.save("testing_data_gy.npy",self.gy_test_data)
        np.save("testing_data_gz.npy",self.gz_test_data)
        
        
        np.save("testing_labels.npy",self.test_labels)
        
        
        
        
        
        #REBUILD_DATA = False
        #print(self.all_data[0])

        
     
        
      

        
       
                
       

if REBUILD_DATA:
    
    HAR = HAR()
    HAR.data_processing()
   

